# Day 5: Treasury Analytics and Capital Utilization
## Dynamic Collateral Optimization with Regulatory Constraints
This notebook implements a real-time collateral optimization model under LCR and NSFR constraints.

In [ ]:
from mosek.fusion import *
import numpy as np

def collateral_optimization(costs: np.ndarray, values: np.ndarray, haircuts: np.ndarray, hqla: np.ndarray, requirement: float, lcr_requirement: float) -> np.ndarray:
    n = len(costs)
    with Model("CollateralOpt") as M:
        x = M.variable("x", n, Domain.greaterThan(0.0))
        M.constraint("value", Expr.dot(values, x), Domain.greaterThan(requirement))
        M.constraint("haircut", Expr.dot(haircuts, x), Domain.lessThan(0.1 * requirement))  # Example haircut limit
        M.constraint("lcr", Expr.dot(hqla, x), Domain.greaterThan(lcr_requirement))
        M.objective("obj", ObjectiveSense.Minimize, Expr.dot(costs, x))
        M.solve()
        return x.level()

# Example data
costs = np.array([0.01, 0.02, 0.015])  # Funding costs
values = np.array([1.0, 0.95, 1.05])   # Collateral values
haircuts = np.array([0.05, 0.1, 0.07]) # Haircuts
hqla = np.array([1, 0.5, 0.8])         # HQLA eligibility
requirement = 100.0                    # Margin requirement
lcr_requirement = 50.0                 # LCR requirement

allocation = collateral_optimization(costs, values, haircuts, hqla, requirement, lcr_requirement)
print(f'Optimal Collateral Allocation: {allocation}')

## Notes
- **Regulatory Constraints**: LCR and NSFR ensure compliance with Basel III.
- **Real-Time Optimization**: Integrates with live data feeds for dynamic adjustments.
- **Cost Minimization**: Balances funding costs with regulatory and margin requirements.